In [1]:
import sys
import os
from PyQt5.QtWidgets import *
from PyQt5 import uic
from PyQt5.QtCore import *
import requests
import json
from PyQt5.QtGui import *
import urllib.request


def resposeSetting(self,responseAuthorization , queryData, dirUrl):            
    #이미지 검색

    url ="https://dapi.kakao.com/v2/search/image"

    headers = {
        "Authorization" : "KakaoAK " + responseAuthorization
    }

    data = {
        "query": queryData
    } 

    #이미지 검색요청

    return requests.post(url, headers = headers, data= data)
    

class ThreadResponse(QThread):
    
    change_value = pyqtSignal(int)
    change_img = pyqtSignal(str)
    
    #초기화 메서드 구현 
    def __init__(self, parent): #parent는 WndowClass에서 전달하는 self이다.(WidnowClass의 인스턴스) 
        super().__init__(parent) 
        self.parent = parent #self.parent를 사용하여 WindowClass 위젯을 제어할 수 있다. 
        
    def save_image(self,image_url, file_name):
        img_response = requests.get(image_url)
        #요청에 성공했다면
        if img_response.status_code == 200:
            #파일저장
            with open(file_name,"wb") as fp:
                fp.write(img_response.content)

    def run(self):#쓰레드로 동작시킬 함수 내용 구현
        count =1

        for image_info in response.json()['documents']:

            file_name = "%d.jpg" %(count)

            #이미지 저장
            self.save_image(image_info['image_url'],dirUrl +"/"+file_name)
            
            self.change_img.emit(image_info['image_url'])
            self.change_value.emit(count)
            self.parent.textBrowser.append(image_info['image_url'] + dirUrl +"/"+ file_name)
            count = count+1
            
#UI파일 연결
#단, UI파일은 Python 코드 파일과 같은 디렉토리에 위치해야한다.
form_class = uic.loadUiType("main.ui")[0]

#화면을 띄우는데 사용되는 Class 선언
class WindowClass(QMainWindow, form_class) :
    def __init__(self) :
        super().__init__()
        self.setupUi(self)
        self.th = ThreadResponse(self)
        
        self.dirButton.clicked.connect(self.dirButtonPushed)
        self.mkdirButton.clicked.connect(self.mkdirButtonPushed)
        self.saveButton.clicked.connect(self.saveButtonPushed)
        self.th.change_value.connect(self.progressBar.setValue)
        self.th.change_img.connect(self.sampleImageChange)
        
    def mkdirButtonPushed(self):
        directory = self.selectQueryEdit.text()
        try:
            if not os.path.exists(directory):
                os.makedirs(directory)
            self.dirLineEdit.setText(os.getcwd() +"/"+ directory)
        except OSError:
            print ('Error: Creating directory. ' +  directory)
        
    def dirButtonPushed(self):
        working_path = str(QFileDialog.getExistingDirectory(self, "Select Directory"))
        self.dirLineEdit.setText(working_path)
    
    def sampleImageChange(self,urlString):
        qPixmapVar = QPixmap()
        imageFromWeb = urllib.request.urlopen(urlString).read()
        qPixmapVar.loadFromData(imageFromWeb )
        self.sampleImage.setPixmap(qPixmapVar.scaledToHeight(210))
    
    def saveButtonPushed(self):
   
        self.textBrowser.clear()
        responseAuthorization = self.apiKeyLineEdit.text()
        global dirUrl
        if self.dirLineEdit.text() =="":
            dirUrl = os.getcwd()
        else:
            dirUrl =self.dirLineEdit.text()
        queryData = self.selectQueryEdit.text()
        global response
        response = resposeSetting(self,responseAuthorization , queryData, dirUrl)
        if response.status_code != 200:
            print("error! because ", response.json())
        else: #성공했다면
            print("Thread시작" )
            
            
            maxCount = len(response.json()['documents'])

            self.progressBar.setRange(0, maxCount)
            self.th.start()
            os.startfile(dirUrl)
            
        
        #self는 WindowClass의 인스턴스, Thread 클래스에서 parent로 전달 
        
        
    
if __name__ == "__main__" :
    #QApplication : 프로그램을 실행시켜주는 클래스
    app = QApplication(sys.argv) 

    #WindowClass의 인스턴스 생성
    myWindow = WindowClass() 

    #프로그램 화면을 보여주는 코드
    myWindow.show()

    #프로그램을 이벤트루프로 진입시키는(프로그램을 작동시키는) 코드
    app.exec_()